In [ ]:
degrees = [x for x in range(5)]

In [ ]:
"""from data_utils import load_ucr_archive, load_ucr_dataset
import seaborn as sns
data = load_ucr_archive(min_ts_length=80,max_ts_length=1000)
ts_lengths = [len(x[1]['data']) for x in data.iterrows()]
print(ts_lengths)
sns.displot(ts_lengths)
"""


In [ ]:
from utils.compression import generate_compression_ratios
from utils.data import load_ucr_data_short

data = load_ucr_data_short()
print("number of time series:", len(data.index))
data.head()
compression_ratios = generate_compression_ratios(data, max(degrees))
print("compression ratios:", compression_ratios)

In [ ]:
%%time
from utils.compression import round_to_nearest_ratio
from utils.spline import generate_coeff_counts, generate_knot_vector_from_coeff_count
from utils.fit import fit_max_spline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from utils.spline import evaluate_spline
import pandas as pd

results = []
error_ts = []
counter = 0
for index, row in data.iterrows():
    time_series = row['data']
    print(row['dataset'], "no.", row['num'], "len(ts):", len(time_series), "ts processed:",
          round(counter / len(data.index) * 100, 1), "%")
    counter += 1
    for degree in degrees:
        counts = generate_coeff_counts(num_data_pts=len(time_series), degree=degree,
                                       compression_ratios=compression_ratios)
        for count in counts:
            knots = generate_knot_vector_from_coeff_count(degree=degree, num_coeffs=count)

            #max_dist, result = fit_max_l1_spline(time_series, knots, degree, eps=0.0000001)
            max_dist, result = fit_max_spline(time_series, knots, degree)

            if max_dist is None and result is None:
                error_ts.append({
                    'dataset': row['dataset'],
                    'num': row['num'],
                    'degree': degree,
                    'num_data_pts': len(time_series),
                    'num_coeffs': count,
                })
                continue

            # calculate error measures
            fitted_spline = [evaluate_spline(knots, result, degree, x[0]) for x in time_series]
            mse = mean_squared_error(fitted_spline, [x[1] for x in time_series])
            mae = mean_absolute_error(fitted_spline, [x[1] for x in time_series])

            results.append({
                'dataset': row['dataset'],
                'num': row['num'],
                'degree': degree,
                'num_data_pts': len(time_series),
                'num_coeffs': count,
                'compression_rate': round_to_nearest_ratio(count / len(time_series), ratios=compression_ratios),
                'max_dist': max_dist,
                'MSE': mse,
                'MAE': mae
            })

            #plot_splines(plt, knots, degree, time_series)
            #plt.show()


In [ ]:
e1_results = pd.DataFrame(results)
error_df = pd.DataFrame(error_ts)
e1_results

# PLOTS

In [ ]:
from utils.plot import plot_errors_against_degrees, plot_errors_against_compression_rates_for_each_degree, plot_errors_against_compression_rates_avg_degree
plot_errors_against_degrees(e1_results)

In [ ]:
plot_errors_against_compression_rates_for_each_degree(e1_results)

In [ ]:
plot_errors_against_compression_rates_avg_degree(e1_results)